In [22]:
import random
import pandas as pd
import numpy as np
import scipy as sp
import nltk
import PyPDF2

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge

The function below tags each text answer for the presence (1) or absence (0) of past tense words as classified by the nltk package.  Past tense is one of the many indices used in classic LIWC studies (e.g., Pennebaker & Francis, 1996/1999), which have been shown to significantly predict personality traits.  We cannot use the LIWC library here as it is not open access, but we can approximate it using the nltk package.

In [84]:
def pastTag(variable):
    for row in variable:
        if "VBD" or "VBN" in nltk.pos_tag(row):
            past = 1
        else:
            past = 0
        return past 
         

The presentTag() function tags each text answer for the presence (0) or absence (1) of present tense words as classified by the nltk package.  Is is also an approximation to the LIWC measure of present tense.  Fewer present tense words have been found to be predictive of openness to experience.

In [87]:
def presentTag(variable):
    for row in variable:
        if "VBG" or "VBP" or "VPZ" in nltk.pos_tag(row):
            present = 1
        else:
            present = 0
        return present

The wordLengths function measures the average word length of each text answer and then returns the percentage of words that have five or more letters.  Having a higher percentage of words greater than five letters long has been found to be a good predictor of openness to experience.

In [61]:
def wordLengths(variable):
    for row in variable:
        lengths = list(map(len,row.split()))
        sumlengths = sum(i > 5 for i in lengths)
        #print(sumlengths)
        percentoverfive = sumlengths/len(row)*100
        return percentoverfive

Negative affect is another index known to predict traits such as neuroticism (positively) and extraversion, agreeableness, and conscientiousness (negatively). We chose to use a publicly available library 

In [89]:
file = open('LM_Negative.pdf', 'rb')

# creating a pdf reader object
fileReader = PyPDF2.PdfFileReader(file)
pageObj = fileReader.getPage(0)
page_content = pageObj.extractText()
#print(page_content)

In [90]:
negativewords = ""
for page in range(fileReader.numPages):
    pageObj = fileReader.getPage(page)
    page_content = pageObj.extractText()
    negativewords = negativewords + page_content
#print(negativewords)
negwords = negativewords.split()

negwords = negwords[7:]

Percentage of negative words: Ok, this one is not working either.  "Negative" seems not to be translating to the dataset correctly.  Any way you could check it out to see what's wrong?

In [79]:
def negAffect(variable):
    for row in variable:
        negative = 0
        words = row.split()
        for word in words:
            if word.upper() in negwords:
                negative = negative + 1
                print(negative)
                percentnegative = negative/len(row)
                return percentnegative
        return percentnegative